In [18]:
!pip install --upgrade pip
!pip install -q scikit-learn==1.4 && pip install -q --no-deps scikeras

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 6.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
D:\PyThon\python.exe -m pip install --upgrade pip
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires scikit-learn>=1.4.2, but you have scikit-learn 1.4.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data/bx/op/com'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
train = pd.read_csv(r'D:\mmmm\pycharmM\apr17\data\bx\op\com\train.csv')
test = pd.read_csv(r'D:\mmmm\pycharmM\apr17\data\bx\op\com\test.csv')
original = pd.read_csv(r'D:\mmmm\pycharmM\apr17\data\bx\op\com\train.csv')

In [21]:
train = pd.concat([train, original]).reset_index(drop=True) 
train = train.drop_duplicates(keep="last").reset_index(drop=True)
train = train.sample(frac=1).reset_index(drop=True)

print('Size of train dataset: {} ____ Size of test dataset: {}'.format(train.shape,test.shape))

Size of train dataset: (381109, 12) ____ Size of test dataset: (127037, 11)


In [22]:
# Define column
less = []
for i in train.columns[1:]:
    if train[i].nunique() < 10:
        less.append(i)
        
print('Column have LESS than 10 unique values: ', less)
print('Column have MORE than 10 unique values: ', [i for i in train.columns if i not in less])

for i in less:
    train[i] = train[i].astype('category')

Column have LESS than 10 unique values:  ['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Response']
Column have MORE than 10 unique values:  ['id', 'Age', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']


In [23]:
from pandas.api.types import CategoricalDtype

# Define the new order of categories
new_categories = ['< 1 Year', '1-2 Year', '> 2 Years']

# Create a new CategoricalDtype with the desired order
new_dtype = CategoricalDtype(categories=new_categories, ordered=True)

# Update the 'Vehicle_Age' column with the new dtype
train['Vehicle_Age'] = train['Vehicle_Age'].astype(new_dtype)

In [24]:
train['age_bins'] = pd.cut(train['Age'], bins=7)
train['Age_Type'] = train['age_bins'].cat.codes
train['Age_x_Vehicle_Age'] = train['Age_Type'] * train['Vehicle_Age'].cat.codes
train['Age_x_Vehicle_Damage'] = train['Age_Type'] * train['Vehicle_Damage'].cat.codes
train['Age_x_Previously_Insured'] = train['Age_Type'] * train['Previously_Insured'].cat.codes
train

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,age_bins,Age_Type,Age_x_Vehicle_Age,Age_x_Vehicle_Damage,Age_x_Previously_Insured
0,175302,Female,59,1,46.0,0,1-2 Year,No,40588.0,11.0,209,0,"(57.143, 66.429]",4,4,0,0
1,132243,Male,43,1,28.0,0,1-2 Year,Yes,25844.0,3.0,179,1,"(38.571, 47.857]",2,2,2,0
2,284074,Female,27,1,3.0,1,< 1 Year,No,39036.0,152.0,151,0,"(19.935, 29.286]",0,0,0,0
3,247131,Male,59,1,28.0,0,1-2 Year,No,40964.0,26.0,161,0,"(57.143, 66.429]",4,4,0,0
4,258841,Female,42,1,28.0,1,1-2 Year,No,42538.0,26.0,235,0,"(38.571, 47.857]",2,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381104,59630,Male,71,1,8.0,0,1-2 Year,Yes,47261.0,26.0,78,0,"(66.429, 75.714]",5,5,5,0
381105,273228,Male,22,1,15.0,1,< 1 Year,No,31996.0,152.0,275,0,"(19.935, 29.286]",0,0,0,0
381106,348608,Male,21,1,8.0,0,< 1 Year,Yes,33815.0,160.0,190,0,"(19.935, 29.286]",0,0,0,0
381107,268917,Male,46,1,28.0,0,> 2 Years,Yes,35043.0,55.0,133,1,"(38.571, 47.857]",2,4,2,0


In [25]:
X = train.drop(['id','Response','age_bins'],axis=1)
y = train['Response']

In [26]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

coltrans = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['Gender', 'Vehicle_Damage']),
        ('minmax', MinMaxScaler(), ['Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Policy_Sales_Channel', 'Vintage']),
        ('ordinal', OrdinalEncoder(categories=[['< 1 Year', '1-2 Year', '> 2 Years']]), ['Vehicle_Age']),
        ('robust', RobustScaler(), ['Annual_Premium']),
        ('standard', StandardScaler(), ['Age_Type', 'Age_x_Vehicle_Age', 'Age_x_Vehicle_Damage', 'Age_x_Previously_Insured'])
    ],
    remainder='passthrough'  # Keeps columns not specified in transformers
)

ImportError: cannot import name 'check_pandas_support' from 'sklearn.utils' (D:\PyThon\Lib\site-packages\sklearn\utils\__init__.py)

In [ ]:
# # Column Transformer
# X_train_trans = coltrans.fit_transform(X_train)
# X_test_trans = coltrans.transform(X_test)

In [27]:
for i in less[:-1]:
    test[i] = train[i].astype('category')
from pandas.api.types import CategoricalDtype

# Define the new order of categories
new_categories = ['< 1 Year', '1-2 Year', '> 2 Years']

# Create a new CategoricalDtype with the desired order
new_dtype = CategoricalDtype(categories=new_categories, ordered=True)

# Update the 'Vehicle_Age' column with the new dtype
test['Vehicle_Age'] = test['Vehicle_Age'].astype(new_dtype)
test['age_bins'] = pd.cut(test['Age'], bins=7)
test['Age_Type'] = test['age_bins'].cat.codes
test['Age_x_Vehicle_Age'] = test['Age_Type'] * test['Vehicle_Age'].cat.codes
test['Age_x_Vehicle_Damage'] = test['Age_Type'] * test['Vehicle_Damage'].cat.codes
test['Age_x_Previously_Insured'] = test['Age_Type'] * test['Previously_Insured'].cat.codes
test.drop(['id','age_bins'],axis=1,inplace=True)

# # Column Transformer
# test_trans = coltrans.transform(test)

In [28]:
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, RobustScaler, StandardScaler
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import make_pipeline
from sklearn.base import clone
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
import keras
import gc

ModuleNotFoundError: No module named 'tensorflow'

In [29]:
TARGET = 'Response'
N_FOLDS = 5
SEED = 42

N_EPOCHS = 30
LEARNING_RATE = 0.005
BATCH_SIZE = 2048
EVAL_BATCH_SIZE = 65536

In [30]:
class Trainer:
    def __init__(self, model, n_folds=N_FOLDS):
        self.model = model
        self.n_folds = n_folds

    def fit_predict(self, X, y, X_test):
        print(f'Training {self.model.__class__.__name__}\n')
        
        scores = []        
        oof_pred_probs = np.zeros((X.shape[0], len(np.unique(y))))
        test_pred_probs = np.zeros((X_test.shape[0], len(np.unique(y))))
        
        skf = StratifiedKFold(n_splits=self.n_folds, random_state=SEED, shuffle=True)
        for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            model = clone(self.model)
            model.fit(X_train, y_train)
            
            y_pred_probs = model.predict_proba(X_val)
            oof_pred_probs[val_idx] = y_pred_probs            
            score = roc_auc_score(y_val, y_pred_probs[:, 1])
            scores.append(score)
            
            temp_test_pred_probs = model.predict_proba(X_test)
            test_pred_probs += temp_test_pred_probs / self.n_folds
            
            print(f'\n--- Fold {fold_idx + 1} - AUC: {score:.5f}\n\n')
            
            del model, X_train, y_train, X_val, y_val, y_pred_probs, temp_test_pred_probs
            gc.collect()
            
        self._save_pred_probs(oof_pred_probs, np.mean(scores), 'oof')
        self._save_pred_probs(test_pred_probs, np.mean(scores), 'test')
        self._save_submission(test_pred_probs, np.mean(scores))
        
        print(f'------ Average AUC:      {np.mean(scores):.5f} ± {np.std(scores):.5f}\n\n')
        
        return oof_pred_probs, scores
        
    def _save_pred_probs(self, pred_probs, cv_score, name):
        model_name = self.model.__class__.__name__.lower().replace('classifier', '')
        with open(f'{model_name}_{name}_pred_probs_{cv_score:.5f}.pkl', 'wb') as f:
            pickle.dump(pred_probs, f)
    
    def _save_submission(self, test_pred_probs, score):
        name = self.model.__class__.__name__.lower().replace('classifier', '')
        sub = pd.read_csv('/kaggle/input/playground-series-s4e7/sample_submission.csv')
        sub['id'] = sub['id']
        sub[TARGET] = test_pred_probs[:, 1]
        # Referencee https://www.kaggle.com/code/paddykb/a-glitch-in-the-insurance-matrix
        INPUT_DIR = Path('data/bx/op/com')
        train_data = pd.read_csv(INPUT_DIR / 'train.csv')
        test_data = pd.read_csv(INPUT_DIR / 'test.csv')
        orig_data = pd.read_csv(r'D:\mmmm\pycharmM\apr17\data\bx\op\com\train.csv')
        features = sorted(set(test_data.columns) - set(['id']))
        train_data.merge(orig_data, on=features).filter(['Response_x', 'Response_y']).value_counts().reset_index()
        override_sub = test_data.merge(orig_data.drop(columns=['id']), on=features).assign(override=lambda x: np.where(x['Response'] == 0, 1, 0)).filter(['id', 'override']).groupby(['id'], as_index=False).agg(override=('override', 'mean'))
        sub.merge(override_sub, how='outer').assign(Response=lambda x: np.where(x['override'].isna(), x['Response'], x['override'])).filter(['id', 'Response']).to_csv(f'sub_{name}_{score:.5f}.csv', index=False)

In [31]:
pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), ['Gender', 'Vehicle_Damage']),
            ('minmax', MinMaxScaler(), ['Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Policy_Sales_Channel', 'Vintage']),
            ('ordinal', OrdinalEncoder(categories=[['< 1 Year', '1-2 Year', '> 2 Years']]), ['Vehicle_Age']),
            ('robust', RobustScaler(), ['Annual_Premium']),
            ('standard', StandardScaler(), ['Age_Type', 'Age_x_Vehicle_Age', 'Age_x_Vehicle_Damage', 'Age_x_Previously_Insured'])
        ],
        remainder='passthrough'  # Keeps columns not specified in transformers
    )
)

X = pipeline.fit_transform(X, y)
X_test = pipeline.transform(test)

del train, test, pipeline
gc.collect()

NameError: name 'make_pipeline' is not defined

In [ ]:
def get_model(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(52, kernel_initializer='truncated_normal', activation='silu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(660, kernel_initializer='truncated_normal', activation='silu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(508, kernel_initializer='truncated_normal', activation='silu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, kernel_initializer='truncated_normal', activation='sigmoid'))
    return model

In [ ]:
model = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    metrics=[keras.metrics.AUC(name='auc')],
    optimizer=keras.optimizers.AdamW(learning_rate=LEARNING_RATE),
    validation_split=0.1,
    batch_size=BATCH_SIZE,
    validation_batch_size=EVAL_BATCH_SIZE,
    epochs=N_EPOCHS, 
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor='val_auc', patience=3, factor=0.3),
        keras.callbacks.EarlyStopping(monitor='val_auc', patience=3)
    ]
)

In [ ]:
trainer = Trainer(model)
oof_pred_probs, scores = trainer.fit_predict(X, y, X_test)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))

fpr, tpr, _ = roc_curve(y, oof_pred_probs[:, 1])
axes[0].plot(fpr, tpr, label=f'AUC: {np.mean(scores):.5f}', color='#651FFF')
axes[0].plot([0, 1], [0, 1], color='black', linestyle='--')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].legend()
axes[0].grid(True)
axes[0].set_aspect('equal', adjustable='box')

y_pred = oof_pred_probs.argmax(axis=1)
cm = confusion_matrix(y, y_pred)
sns.heatmap(
    cm, 
    annot=True, 
    fmt=',', 
    ax=axes[1], 
    cbar=False,
    cmap=sns.light_palette('#651FFF', reverse=False, as_cmap=True), 
)
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()